In [1]:
import os
import re
import json
import glob
import spacy
import pickle
import random
import difflib
import textwrap
import datetime
import jsonlines
import numpy as np
import pandas as pd
from utils.shared_configs import LLAMA_MODEL_PATH, ZEHPYR_MODEL_PATH, get_sampling_params, initialize_llm

2025-04-27 22:02:07.045293: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 22:02:07.062351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745791327.083502   95470 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745791327.090029   95470 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 22:02:07.112018: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
llm = initialize_llm(model_path=LLAMA_MODEL_PATH, tokenizer_path=LLAMA_MODEL_PATH)

In [ ]:
sampling_params = get_sampling_params(max_tokens=300, temperature=0.6, top_p=0.9, stops=["</s>", "\n}"])

In [94]:
# with open(f"outputs/pointwise/pointwise_iteration_curr/pointwise_sed_outputs.json", "r", encoding="utf-8") as f:
#     pointwise_sed_outputs=json.load(f)

In [ ]:
# entity = pointwise_sed_outputs[0]['entities'][0]
# entity

In [2]:
wn_val_subset = pd.read_csv('wn_val_subset_4_27_pointwise_results.csv')
wn_val_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           100 non-null    int64  
 1   article_text         100 non-null    object 
 2   date                 100 non-null    object 
 3   article_title        100 non-null    object 
 4   entity_title         100 non-null    object 
 5   entity_salience      100 non-null    int64  
 6   offsets              100 non-null    object 
 7   url                  100 non-null    object 
 8   surrounding_context  100 non-null    object 
 9   gt_wiki_id           99 non-null     float64
 10  candidates           95 non-null     object 
dtypes: float64(1), int64(2), object(8)
memory usage: 8.7+ KB


In [3]:
COT_POOL = [
    {
        "mention": "Apple",
        "left_context": "Tim Cook is the CEO of",
        "right_context": "which released the iPhone 15.",
        "candidates": [
            {"id": 1, "name": "Apple Inc.", "summary": "An American consumer‑electronics company."},
            {"id": 2, "name": "Apple (fruit)", "summary": "An edible fruit."},
        ],
        "answer": "1.Apple Inc."
    },
    {
        "mention": "Jaguar",
        "left_context": "He bought a new",
        "right_context": "to replace his old sedan.",
        "candidates": [
            {"id": 9, "name": "Jaguar Cars", "summary": "A British luxury‑car brand."},
            {"id": 8, "name": "Jaguar (animal)", "summary": "A large cat native to the Americas."},
        ],
        "answer": "9.Jaguar Cars"
    }
]

In [4]:
    INSTRUCTION_PROMPT = "1. Context: Look at the surrounding text to understand the topic.\n2. Categories: Consider the type of the entity (person, organization, location, etc.).\n3. Modifiers: Pay attention to words or phrases that add details to the mention.\n4. Co-references: Check other mentions of the same entity in the text.\n5. Temporal and Geographical Factors: Consider when and where the text was written.\n6. External Knowledge: Use knowledge from outside the text.\nRemember, effective entity disambiguation requires understanding the text thoroughly, having world knowledge, and exercising good judgment.\n"

In [3]:
def contextual_el_prompt(entity, candidates, CoT_POOL=COT_POOL, INSTRUCTION_PROMPT=INSTRUCTION_PROMPT):
    ex = random.choice(COT_POOL)
    
    # CoT exemplar
    ex_ctx = f"{ex['left_context']} ###{ex['mention']}### {ex['right_context']}"
    ex_block = textwrap.dedent(f"""\
        The following example illustrates the task:
        Mention: {ex['mention']}
        Context: {ex_ctx}
        Candidates: {ex['candidates']}
        Answer: {ex['answer']}""")
    
    # create candidate map for later reconcilation
    # cand_map, target_lines = {}, []
    # for idx, cand in enumerate(random.sample(entity['candidates'], len(entity['candidates'])), 1):
    #     cand_map[idx] = cand
    #     target_lines.append(f"Entity {idx}: {cand['cand_name']}. {cand['cand_summary']}")
    label_map = {}                         # label → candidate‑dict
    cand_lines = []
    for lbl, cand in enumerate(random.sample(entity['candidates'], len(entity['candidates'])), 1):
        cand_lines.append(f"{lbl}. {cand['cand_name']} – {cand['cand_summary']}")
        label_map[lbl] = cand              # store mapping
        cand['prompt_label'] = lbl         # optional: keep inside dict

    tgt_ctx = (entity['left_context'].strip() + ' ###' + entity['entity_title'] + '### ' + entity['right_context'].strip())
    tgt_cand_lines = []
    for idx, cand in enumerate(random.sample(entity['candidates'], len(entity['candidates'])), 1):
        tgt_cand_lines.append(f"{idx}. {cand['cand_name']} – {cand['cand_summary']}")

    tgt_block = textwrap.dedent(f"""\
        Now I will give you a new mention, its context, and a list of candidate entities.
        The mention is highlighted with '###'.

        Mention: {entity['entity_title']}
        Context: {tgt_ctx}
        {'; '.join(tgt_cand_lines)}

        Think step by step.  At the end output exactly one line with the ID
        and name of the chosen entity, e.g.  '3.Barack Obama'.
        If none fit, output '-1.None'.
    """)
    
    prompt = f"""
        <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        {SYSTEM_PROMPT}<|eot_id|><|start_header_id|>user<|end_header_id|>
        {INSTRUCTION_PROMPT}{ex_block}{tgt_block}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
        """.strip()
    
    return prompt.strip(), label_map

In [96]:
MAX_PROMPTS = 7500
# (art_idx, ent_idx, cand_map, prompt)
cx_prompt_records = []
count=0
for art_idx, art in enumerate(pointwise_sed_outputs):
    for ent_idx, ent in enumerate(art["entities"]):
        if not ent.get("candidates"):
            count+=1
            continue

        if len(cx_prompt_records) >= MAX_PROMPTS:
            break

        prompt, cmap = contextual_el_prompt(ent, ent.get("candidates"))
        cx_prompt_records.append((art_idx, ent_idx, cmap, prompt))

    if len(cx_prompt_records) >= MAX_PROMPTS:
        break
        
print(f"Collected {len(cx_prompt_records)} prompts "
      f"(last = {cx_prompt_records[-1][:2]})")

Collected 731 prompts (last = (272, 6))


In [97]:
count

26529

In [98]:
cx_prompt_records[0]

(0,
 0,
 {1: {'mention': 'United States',
   'cand_name': 'United States',
   'cand_summary': "The United States of America is a federal republic and representative democracy, comprising 50 states, five major self-governing territories, and various possessions, with an estimated population of over 327 million people, the third most populous country in the world, and a highly developed country with the world's largest economy by nominal GDP, second-largest by purchasing power parity, and a leading political, cultural, and scientific force internationally.",
   'cand_text': "The United States of America (USA), commonly known as the United States (U.S. or US) or simply America, is a country comprising 50 states, a federal district, five major self-governing territories, and various possessions. At 3.8&nbsp;million square miles (9.8&nbsp;million km<sup>2</sup>), it is the world's third or fourth largest country by total area and is slightly smaller than the entire continent of Europe. Most

In [99]:
outputs = llm.generate(prompts=prompts, sampling_params=sampling_params)

Processed prompts: 100%|██████████| 731/731 [02:32<00:00,  4.80it/s, est. speed input: 2987.28 toks/s, output: 1139.74 toks/s]


In [101]:
# answer_pat = re.compile(r'(-?\d+)\s*\.(.+)')  
# for rec, out in zip(cx_prompt_records, outputs):
#     art_idx, ent_idx, cmap, _ = rec
#     text = out.outputs[0].text.strip()

#     # --- grab the first "<id>.<name>" we see (top‑to‑bottom)
#     m = answer_pat.search(text)
#     if not m:
#         chosen_id = -1
#     else:
#         chosen_id = int(m.group(1))

#     # --- write back
#     ent = pointwise_sed_outputs[art_idx]["entities"][ent_idx]
#     if chosen_id in cmap:                   # LLM picked a valid label
#         ent["candidates"] = [cmap[chosen_id]]
#         ent["linker_response"] = text       # (optional, for inspection)
#     else:                                   # -1.None  or invalid label
#         ent["candidates"] = []
#         ent["linker_response"] = text

answer_pat = re.compile(r'(-?\d+)\s*\.\s*(.+)', re.I)   # e.g.  3.Barack Obama

for record, output in zip(cx_prompt_records, outputs):
    art_idx, ent_idx, label_map, _ = record
    ent   = pointwise_sed_outputs[art_idx]["entities"][ent_idx]

    # raw text
    linker_resp = output.outputs[0].text.strip()
    ent["linker_response"] = linker_resp

    # default: no prediction
    ent["top_linked_entity"] = None

    # grab the last non‑empty line and parse "<label>.<name>"
    lines = [ln.strip() for ln in linker_resp.splitlines() if ln.strip()]
    m     = answer_pat.search(lines[-1]) if lines else None
    if not m:                         # failed to parse → leave None
        continue

    lbl = int(m.group(1))
    if lbl < 0 or lbl not in label_map:
        continue                      # "-1.None" or invalid label

    cand = label_map[lbl]             # candidate dict chosen by the model
    ent["top_linked_entity"] = {
        "cand_name":    cand["cand_name"],
        "cand_wiki_id": cand["cand_wiki_id"]
    }

In [102]:
with open(f"outputs/pointwise/pointwise_iteration_curr/contextual_el_sed_outputs_v2.json", "w", encoding="utf-8") as f:
    json.dump(pointwise_sed_outputs, f, indent=2)

In [109]:
def evaluate_linking(data):
    total = 0        # every entity
    linked = 0        # entities with a prediction
    gts = 0        # entities that have a gold wiki_ID
    correct = 0

    for art in data:
        for ent in art["entities"]:
            total += 1
            gt = ent.get("entity wiki_ID") or None
            if gt:
                gts += 1

            pred_id = (ent.get("top_linked_entity") or {}).get("cand_wiki_id")
            if pred_id:
                linked += 1

            if gt and pred_id and gt == pred_id:
                correct += 1

    accuracy  = correct / total if total else 0
    precision = correct / linked if linked else 0
    recall    = correct / gts if gts else 0

    print(f"Entities evaluated : {total}")
    print(f"Ground truths non‑null : {gts}")
    print(f"Predictions made : {linked}")
    print(f"Correct links : {correct}\n")

    print(f"Accuracy : {accuracy:.4%}")
    print(f"Precision @ linked : {precision:.4%}")
    print(f"Recall : {recall:.4%}")

evaluate_linking(pointwise_sed_outputs)

Entities evaluated : 27260
Ground truths non‑null : 22623
Predictions made : 623
Correct links : 161

Accuracy : 0.5906%
Precision @ linked : 25.8427%
Recall : 0.7117%
